[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](<https://colab.research.google.com/github/MauPos04/AI_NOTEBOOKS/blob/main/Web%20Scrap%20and%20Translate%20Using%20Transformers.ipynb>)

# Web Scraping y Aplicación de Transformers

## WebScraping

In [ ]:
%pip install requests
%pip install beautifulsoup4

In [ ]:
# importacion de librerias
from bs4 import BeautifulSoup
import requests

In [ ]:
# URL de la pagina a scrapear
URL = "https://www.diariobitcoin.com/ia/alibaba-redefine-ia-nuevo-modelo-qwq-32b-optimiza-rendimiento-en-ia-con-menos-recursos/"
page = requests.get(URL)

In [ ]:
print(page.text)

In [ ]:
# Creacion de un objeto BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
# Extraccion de los elementos de la pagina
results = soup.find_all("div", class_="post-content")

In [ ]:
# iteración para seleccionar los resultados de la búsqueda (título, empresa y ubicación)
for new_element in results:
  news_element = new_element.find("div", class_="entry-content")

In [ ]:
# extracción del texto de la noticia a partir de la etiqueta <hr/>
news_element_str = str(news_element)

soup = BeautifulSoup(news_element_str, 'html.parser')
hr_tag = soup.find('hr')
content_after_hr = hr_tag.find_all_next(string=True)
extracted_text = '\n'.join(content_after_hr)

In [ ]:
# limpieza del texto extraído de la noticia y eliminación de la marca de agua
final_text = 'Imagen original de DiarioBitcoin'
Text = extracted_text.strip().replace("\n", "").split(final_text)[0]
print(Text)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

## Summarize

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'csebuetnlp/mT5_multilingual_XLSum'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt).to(device)

def generate_summary_csebuetnlp(text):
    inputs = tokenizer(
        [text],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
generate_summary_csebuetnlp(Text)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt).to(device)

def generate_summary_bart_large(text):
    inputs = tokenizer(
        [text],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    output = model.generate(input_ids, attention_mask=attention_mask)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
generate_summary_bart_large(Text)

## Translation

In [ ]:
model_id = "ModelSpace/GemmaX2-28-2B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

text = f"Translate this from Spanish to English:\nSpanish: {Text}\nEnglish:"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))